# LoRaWAN Session Data Analyzer

Dieses Notebook ermöglicht die interaktive Analyse von LoRaWAN Session Daten.

## Datenstruktur
Die CSV-Dateien enthalten folgende Spalten:
- `timestamp`: Zeitstempel der Nachricht
- `session_id`: Session-ID
- `device_eui`: Device Identifier
- `fcnt`: Frame Counter
- `rssi_dbm`: Signal Strength (dBm)
- `snr_db`: Signal-to-Noise Ratio (dB)
- `spreading_factor`: LoRaWAN Spreading Factor
- `frequency`: Übertragungsfrequenz
- `device_lat/lon`: GPS-Koordinaten des Devices
- `decoded_payload`: Sensor-Daten (JSON)
- und weitere...

In [1]:
# Importiere benötigte Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from datetime import datetime
from glob import glob
import warnings

# Konfiguration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Bibliotheken erfolgreich importiert!")

Bibliotheken erfolgreich importiert!


In [2]:
# Funktionen für Datenanalyse

def load_and_process_data(file_path):
    """Lädt und verarbeitet die CSV-Datei"""
    df = pd.read_csv(file_path)  # CSV mit Komma-Separator
    
    # Datentyp-Konvertierungen
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['rssi_dbm'] = pd.to_numeric(df['rssi_dbm'], errors='coerce')
    df['snr_db'] = pd.to_numeric(df['snr_db'], errors='coerce')
    df['frequency'] = pd.to_numeric(df['frequency'], errors='coerce')
    df['device_lat'] = pd.to_numeric(df['device_lat'], errors='coerce')
    df['device_lon'] = pd.to_numeric(df['device_lon'], errors='coerce')
    df['fcnt'] = pd.to_numeric(df['fcnt'], errors='coerce')
    
    # Parse JSON payload wenn vorhanden
    if 'decoded_payload' in df.columns:
        df['sensor_data'] = df['decoded_payload'].apply(parse_sensor_data)
    
    return df

def parse_sensor_data(payload_str):
    """Parst die JSON-Payload und extrahiert Sensordaten"""
    try:
        if pd.isna(payload_str) or payload_str == '':
            return None
        
        # Clean the JSON string
        clean_payload = payload_str.replace('""', '"')
        data = json.loads(clean_payload)
        
        result = {}
        if 'd' in data and isinstance(data['d'], list) and len(data['d']) >= 2:
            result['temperature'] = data['d'][0]
            result['humidity'] = data['d'][1]
        
        if 'sf' in data:
            result['sf_from_payload'] = data['sf']
        
        if 't' in data:
            result['time_counter'] = data['t']
        
        return result
    except (json.JSONDecodeError, KeyError, TypeError):
        return None

def get_available_files():
    """Zeigt verfügbare CSV-Dateien an"""
    csv_files = glob("*.csv")
    csv_files.sort()
    return csv_files

def display_file_info(files):
    """Zeigt Informationen über verfügbare Dateien"""
    print("Verfügbare LoRaWAN Session Dateien:")
    print("=" * 50)
    for i, file in enumerate(files):
        file_size = os.path.getsize(file) / 1024  # KB
        mod_time = datetime.fromtimestamp(os.path.getmtime(file))
        print(f"{i+1:2d}. {file}")
        print(f"     Größe: {file_size:.1f} KB, Erstellt: {mod_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print()

print("Hilfsfunktionen definiert!")

Hilfsfunktionen definiert!


In [3]:
# Zeige verfügbare Dateien
available_files = get_available_files()
display_file_info(available_files)

Verfügbare LoRaWAN Session Dateien:
 1. lorawan_data.csv
     Größe: 14.2 KB, Erstellt: 2025-07-07 23:18:09

 2. lorawan_session_20250707_204252_526570b6.csv
     Größe: 0.6 KB, Erstellt: 2025-07-07 23:18:09

 3. lorawan_session_20250707_204441_af09a252 copy.csv
     Größe: 0.3 KB, Erstellt: 2025-07-08 12:29:59

 4. lorawan_session_20250707_210235_7f9c8028 copy.csv
     Größe: 0.9 KB, Erstellt: 2025-07-08 12:29:59

 5. lorawan_session_20250707_210306_bcd879bb copy.csv
     Größe: 0.6 KB, Erstellt: 2025-07-08 12:29:59

 6. lorawan_session_20250707_211812_9c7e7812 copy.csv
     Größe: 0.6 KB, Erstellt: 2025-07-08 12:29:59

 7. lorawan_session_20250707_211833_154b5404 copy.csv
     Größe: 0.3 KB, Erstellt: 2025-07-08 12:29:59

 8. lorawan_session_20250707_211906_f1583e36 copy.csv
     Größe: 0.0 KB, Erstellt: 2025-07-08 12:29:59

 9. lorawan_session_20250707_212614_8d65ca84 copy.csv
     Größe: 9.4 KB, Erstellt: 2025-07-08 12:29:59

10. lorawan_session_20250707_213340_9deb856e copy.csv
  

In [4]:
# DATEIAUSWAHL - Ändern Sie hier die Zahl entsprechend der gewünschten Datei
selected_file_index = 8  # Ändern Sie diese Zahl (1-8) um eine andere Datei zu wählen

if 1 <= selected_file_index <= len(available_files):
    selected_file = available_files[selected_file_index - 1]
    print(f"Ausgewählte Datei: {selected_file}")
    
    # Lade die Daten
    df = load_and_process_data(selected_file)
    print(f"\nDaten erfolgreich geladen: {len(df)} Datensätze")
else:
    print(f"Ungültige Auswahl! Bitte wählen Sie eine Zahl zwischen 1 und {len(available_files)}")

Ausgewählte Datei: lorawan_session_20250707_211906_f1583e36 copy.csv


EmptyDataError: No columns to parse from file

In [ ]:
# Grundlegende Datenübersicht
print("=" * 60)
print("DATENÜBERSICHT")
print("=" * 60)

print(f"Dateiname: {selected_file}")
print(f"Anzahl Datensätze: {len(df)}")
print(f"Zeitraum: {df['timestamp'].min()} bis {df['timestamp'].max()}")
print(f"Dauer: {df['timestamp'].max() - df['timestamp'].min()}")
print(f"Unique Devices: {df['device_eui'].nunique()}")
print(f"Unique Sessions: {df['session_id'].nunique()}")

print("\nSpalten im Datensatz:")
print(df.columns.tolist())

In [ ]:
# Erste Zeilen der Daten anzeigen
print("ERSTE 5 DATENSÄTZE:")
print("=" * 60)
display_columns = ['timestamp', 'fcnt', 'rssi_dbm', 'snr_db', 'spreading_factor', 'frequency', 'device_lat', 'device_lon']
df[display_columns].head()

In [ ]:
# Sensordaten extrahieren und anzeigen
sensor_data_expanded = pd.json_normalize(df['sensor_data'].dropna())

if not sensor_data_expanded.empty:
    print("SENSORDATEN (aus JSON Payload):")
    print("=" * 60)
    print(f"Anzahl Datensätze mit Sensordaten: {len(sensor_data_expanded)}")
    print("\nSensordaten Statistiken:")
    print(sensor_data_expanded.describe())
    
    # Füge Sensordaten zum Haupt-DataFrame hinzu
    df_sensor = df.dropna(subset=['sensor_data']).copy()
    df_sensor = df_sensor.reset_index(drop=True)
    sensor_data_expanded = sensor_data_expanded.reset_index(drop=True)
    
    for col in sensor_data_expanded.columns:
        df_sensor[col] = sensor_data_expanded[col]
    
    print("\nSensordaten wurden zum DataFrame hinzugefügt!")
else:
    print("Keine Sensordaten gefunden.")
    df_sensor = df.copy()

In [ ]:
# Statistische Analyse der Übertragungsqualität
print("ÜBERTRAGUNGSQUALITÄT ANALYSE")
print("=" * 60)

# RSSI Statistiken
rssi_stats = df['rssi_dbm'].describe()
print("RSSI (Received Signal Strength Indicator) Statistiken:")
print(rssi_stats)

# SNR Statistiken
snr_stats = df['snr_db'].describe()
print("\nSNR (Signal-to-Noise Ratio) Statistiken:")
print(snr_stats)

# Spreading Factor Verteilung
print("\nSpreading Factor Verteilung:")
sf_counts = df['spreading_factor'].value_counts().sort_index()
print(sf_counts)

# Frequency Verteilung
print("\nFrequency Verteilung:")
freq_counts = df['frequency'].value_counts().sort_index()
print(freq_counts)

In [ ]:
# Visualisierungen erstellen
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle(f'LoRaWAN Session Analysis - {selected_file}', fontsize=16, fontweight='bold')

# 1. RSSI über Zeit
axes[0, 0].plot(df['timestamp'], df['rssi_dbm'], 'b-', alpha=0.7, linewidth=1.5)
axes[0, 0].set_title('RSSI über Zeit', fontweight='bold')
axes[0, 0].set_xlabel('Zeit')
axes[0, 0].set_ylabel('RSSI (dBm)')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. SNR über Zeit
axes[0, 1].plot(df['timestamp'], df['snr_db'], 'r-', alpha=0.7, linewidth=1.5)
axes[0, 1].set_title('SNR über Zeit', fontweight='bold')
axes[0, 1].set_xlabel('Zeit')
axes[0, 1].set_ylabel('SNR (dB)')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Spreading Factor Verteilung
sf_counts.plot(kind='bar', ax=axes[1, 0], color='green', alpha=0.7)
axes[1, 0].set_title('Spreading Factor Verteilung', fontweight='bold')
axes[1, 0].set_xlabel('Spreading Factor')
axes[1, 0].set_ylabel('Anzahl')
axes[1, 0].tick_params(axis='x', rotation=0)

# 4. RSSI vs SNR Scatter
scatter = axes[1, 1].scatter(df['rssi_dbm'], df['snr_db'], c=df.index, cmap='viridis', alpha=0.7)
axes[1, 1].set_title('RSSI vs SNR Correlation', fontweight='bold')
axes[1, 1].set_xlabel('RSSI (dBm)')
axes[1, 1].set_ylabel('SNR (dB)')
axes[1, 1].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[1, 1], label='Zeitsequenz')

plt.tight_layout()
plt.show()

In [ ]:
# Sensordaten Visualisierung (falls verfügbar)
if 'temperature' in df_sensor.columns and 'humidity' in df_sensor.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Sensordaten Analyse', fontsize=16, fontweight='bold')
    
    # Temperatur über Zeit
    axes[0, 0].plot(df_sensor['timestamp'], df_sensor['temperature'], 'orange', linewidth=2, marker='o', markersize=3)
    axes[0, 0].set_title('Temperatur über Zeit', fontweight='bold')
    axes[0, 0].set_xlabel('Zeit')
    axes[0, 0].set_ylabel('Temperatur (°C)')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Feuchtigkeit über Zeit
    axes[0, 1].plot(df_sensor['timestamp'], df_sensor['humidity'], 'cyan', linewidth=2, marker='o', markersize=3)
    axes[0, 1].set_title('Feuchtigkeit über Zeit', fontweight='bold')
    axes[0, 1].set_xlabel('Zeit')
    axes[0, 1].set_ylabel('Feuchtigkeit (%)')
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Temperatur Histogramm
    axes[1, 0].hist(df_sensor['temperature'], bins=20, alpha=0.7, color='orange', edgecolor='black')
    axes[1, 0].set_title('Temperatur Verteilung', fontweight='bold')
    axes[1, 0].set_xlabel('Temperatur (°C)')
    axes[1, 0].set_ylabel('Häufigkeit')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Feuchtigkeit Histogramm
    axes[1, 1].hist(df_sensor['humidity'], bins=20, alpha=0.7, color='cyan', edgecolor='black')
    axes[1, 1].set_title('Feuchtigkeit Verteilung', fontweight='bold')
    axes[1, 1].set_xlabel('Feuchtigkeit (%)')
    axes[1, 1].set_ylabel('Häufigkeit')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Sensordaten Statistiken
    print("\nSENSORDATEN STATISTIKEN:")
    print("=" * 40)
    print(f"Temperatur: {df_sensor['temperature'].mean():.2f}°C ± {df_sensor['temperature'].std():.2f}°C")
    print(f"Bereich: {df_sensor['temperature'].min():.2f}°C - {df_sensor['temperature'].max():.2f}°C")
    print(f"\nFeuchtigkeit: {df_sensor['humidity'].mean():.2f}% ± {df_sensor['humidity'].std():.2f}%")
    print(f"Bereich: {df_sensor['humidity'].min():.2f}% - {df_sensor['humidity'].max():.2f}%")
else:
    print("Keine Sensordaten (Temperatur/Feuchtigkeit) verfügbar.")

In [ ]:
# Erweiterte Analyse: Korrelationen
print("KORRELATIONSANALYSE")
print("=" * 60)

# Numerische Spalten für Korrelation
numeric_cols = ['rssi_dbm', 'snr_db', 'fcnt', 'frequency']
if 'temperature' in df_sensor.columns:
    numeric_cols.extend(['temperature', 'humidity'])

# Korrelationsmatrix
correlation_data = df_sensor[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_data, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'label': 'Korrelation'})
plt.title('Korrelationsmatrix der Messwerte', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

print("\nStärkste Korrelationen:")
# Finde die stärksten Korrelationen (außer Diagonal)
corr_pairs = []
for i in range(len(correlation_data.columns)):
    for j in range(i+1, len(correlation_data.columns)):
        corr_value = correlation_data.iloc[i, j]
        if not np.isnan(corr_value):
            corr_pairs.append((correlation_data.columns[i], correlation_data.columns[j], corr_value))

# Sortiere nach absoluter Korrelation
corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)

for i, (var1, var2, corr) in enumerate(corr_pairs[:5]):
    print(f"{i+1}. {var1} ↔ {var2}: {corr:.3f}")

In [ ]:
# Netzwerkqualitäts-Dashboard
def analyze_network_quality(df):
    """Analysiert die Netzwerkqualität basierend auf RSSI und SNR"""
    
    # Qualitätskategorien definieren
    def rssi_category(rssi):
        if rssi >= -70:
            return 'Excellent'
        elif rssi >= -80:
            return 'Good'
        elif rssi >= -90:
            return 'Fair'
        else:
            return 'Poor'
    
    def snr_category(snr):
        if snr >= 10:
            return 'Excellent'
        elif snr >= 5:
            return 'Good'
        elif snr >= 0:
            return 'Fair'
        else:
            return 'Poor'
    
    df['rssi_quality'] = df['rssi_dbm'].apply(rssi_category)
    df['snr_quality'] = df['snr_db'].apply(snr_category)
    
    return df

# Analysiere Netzwerkqualität
df_quality = analyze_network_quality(df.copy())

# Visualisiere Qualitätsverteilung
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Netzwerkqualitäts-Dashboard', fontsize=16, fontweight='bold')

# RSSI Qualitätsverteilung
rssi_quality_counts = df_quality['rssi_quality'].value_counts()
colors = ['green', 'orange', 'red', 'darkred']
quality_order = ['Excellent', 'Good', 'Fair', 'Poor']
rssi_quality_counts = rssi_quality_counts.reindex(quality_order, fill_value=0)

axes[0].pie(rssi_quality_counts.values, labels=rssi_quality_counts.index, autopct='%1.1f%%', 
            colors=colors[:len(rssi_quality_counts)], startangle=90)
axes[0].set_title('RSSI Qualitätsverteilung', fontweight='bold')

# SNR Qualitätsverteilung
snr_quality_counts = df_quality['snr_quality'].value_counts()
snr_quality_counts = snr_quality_counts.reindex(quality_order, fill_value=0)

axes[1].pie(snr_quality_counts.values, labels=snr_quality_counts.index, autopct='%1.1f%%', 
            colors=colors[:len(snr_quality_counts)], startangle=90)
axes[1].set_title('SNR Qualitätsverteilung', fontweight='bold')

plt.tight_layout()
plt.show()

# Qualitätsstatistiken ausgeben
print("\nNETZWERKQUALITÄT ZUSAMMENFASSUNG")
print("=" * 60)
print("RSSI Qualitätsverteilung:")
for quality, count in rssi_quality_counts.items():
    percentage = (count / len(df_quality)) * 100
    print(f"  {quality}: {count} Nachrichten ({percentage:.1f}%)")

print("\nSNR Qualitätsverteilung:")
for quality, count in snr_quality_counts.items():
    percentage = (count / len(df_quality)) * 100
    print(f"  {quality}: {count} Nachrichten ({percentage:.1f}%)")

In [ ]:
# Zusammenfassung und Export-Optionen
print("\n" + "=" * 80)
print("ANALYSEZUSAMMENFASSUNG")
print("=" * 80)

summary = {
    'Dateiname': selected_file,
    'Analysezeitpunkt': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'Gesamtnachrichten': len(df),
    'Zeitraum': f"{df['timestamp'].min()} bis {df['timestamp'].max()}",
    'Durchschnittliche_RSSI': f"{df['rssi_dbm'].mean():.2f} dBm",
    'Durchschnittliche_SNR': f"{df['snr_db'].mean():.2f} dB",
    'Häufigster_SF': df['spreading_factor'].mode().iloc[0],
    'Unique_Devices': df['device_eui'].nunique(),
    'Qualität_Excellent_RSSI': f"{(rssi_quality_counts.get('Excellent', 0) / len(df_quality) * 100):.1f}%",
    'Qualität_Excellent_SNR': f"{(snr_quality_counts.get('Excellent', 0) / len(df_quality) * 100):.1f}%"
}

if 'temperature' in df_sensor.columns:
    summary.update({
        'Durchschnittliche_Temperatur': f"{df_sensor['temperature'].mean():.2f}°C",
        'Durchschnittliche_Feuchtigkeit': f"{df_sensor['humidity'].mean():.2f}%"
    })

for key, value in summary.items():
    print(f"{key.replace('_', ' ')}: {value}")

print("\n" + "=" * 80)
print("EXPORT-OPTIONEN")
print("=" * 80)
print("Sie können die Analyseergebnisse exportieren:")
print("1. DataFrame als CSV: df.to_csv('analysis_results.csv')")
print("2. Zusammenfassung als JSON: import json; json.dump(summary, open('summary.json', 'w'))")
print("3. Grafiken als PNG: plt.savefig('analysis_plots.png', dpi=300, bbox_inches='tight')")

print("\nAnalyse abgeschlossen! 🎉")

In [ ]:
# Optionale Detailanalyse - Zeitreihenanalyse
print("ZEITREIHENANALYSE")
print("=" * 60)

# Zeitintervalle zwischen Nachrichten
df_sorted = df.sort_values('timestamp')
time_diffs = df_sorted['timestamp'].diff().dt.total_seconds()
time_diffs = time_diffs.dropna()

print(f"Durchschnittliches Intervall zwischen Nachrichten: {time_diffs.mean():.2f} Sekunden")
print(f"Minimum Intervall: {time_diffs.min():.2f} Sekunden")
print(f"Maximum Intervall: {time_diffs.max():.2f} Sekunden")

# Visualisierung der Zeitintervalle
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(time_diffs, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('Verteilung der Zeitintervalle zwischen Nachrichten', fontweight='bold')
plt.xlabel('Zeitintervall (Sekunden)')
plt.ylabel('Häufigkeit')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(range(len(time_diffs)), time_diffs, 'b-', alpha=0.7)
plt.title('Zeitintervalle über die Session', fontweight='bold')
plt.xlabel('Nachrichtennummer')
plt.ylabel('Zeitintervall (Sekunden)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()